<img src="../../docs/images/DSPy8.png" alt="DSPy7 Image" height="120"/>

### Multi-Agent DSPy Programs: Bootstrapping & Aggregating Multiple `ReAct` Agents

This is a quick (somewhat advanced) example of DSPy. You're given a hard QA task and an agent architecture (`dspy.ReAct`), how do you get high scores without tinkering with prompts?

There are many ways, but this notebook shows one complex strategy that DSPy makes near-trivial to achieve: we'll automatically bootstrap five different highly-effective prompts for ReAct, then optimize an aggregator that combines their powers.

As is usually the case with DSPy, the code to do this is probably shorter than describing it in English, so let's jump right into that.

### 0) TLDR.

We'll build a ReAct agent in DSPy that scores 30% accuracy on a retrieval-based question answering task.

Then, we'll optimize it with `BootstrapFewShotWithRandomSearch` to get 46% accuracy.

Then, we'll build a multi-agent aggregator over five different optimized versions of the agent.

Our unoptimized aggregator will score 26%. It doesn't understand the task. Hence, we'll optimize the aggregator too.

We'll end up with an optimized multi-agent system that scores a whopping 60% accuracy on the same task.

The core portion of the code to do this can be fit into 10 lines of DSPy, but we'll sprinkle some short explanations below.

### 1) Setting Up.

We'll configure the language model (GPT-3.5) and the retrieval model (ColBERTv2 over Wikipedia).

In [1]:
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy

init_sys_env(proxy_url='http://192.168.31.215:10890')
secret_env_file = '/mnt/c/Users/chenzi/Desktop/project/experiment_project/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')

turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=520,api_key=model_config.get('api_key'))
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=turbo, rm=colbert)

### 2) Loading some data.
We'll load 150 examples for training (`trainset`), 50 examples for validation & optimization (`valset`), and 300 examples for evaluation (`devset`).

In [2]:
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=300, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train[0:150]]
valset = [x.with_inputs('question') for x in dataset.train[150:200]]
devset = [x.with_inputs('question') for x in dataset.dev]

# show an example datapoint; it's just a question-answer pair
trainset[0]
# Example({'question': '《在我的窗前》是由哪位美国创作歌手发行的？', 'answer': '约翰·汤斯·范·赞德'}) (input_keys={'question'})

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

/home/cc/miniconda3/envs/py310/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

### 3) ReAct Agent.

Our agent will just be a DSPy ReAct agent that takes a `question` and outputs the `answer` by using a ColBERTv2 retrieval tool.
我们的代理将只是一个DSPy ReAct代理，它通过使用ColBERTv2检索工具获取 `question` 并输出 `answer`。

In [3]:
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

Let's evaluate this **unoptimized** ReAct agent on the `devset`.
---- 让我们在`devset`上评估这个未优化的ReAct代理。

In [5]:
# Set up an evaluator on the first 300 examples of the devset. 在开发集的前300个示例上设置评估器。
config = dict(num_threads=10, display_progress=True, display_table=5)
evaluate = Evaluate(devset=devset, metric=dspy.evaluate.answer_exact_match, **config)

evaluate(agent)

Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)


Average Metric: 10.0 / 109  (9.2):  36%|███▋      | 109/300 [06:11<10:50,  3.40s/it]

Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)




Average Metric: 3 / 29  (10.3):   9%|▉         | 28/300 [00:00<00:00, 494.40it/s]

ValueError: not enough values to unpack (expected 2, got 1)

### 4) Optimized ReAct.

Let's use DSPy's simple `BootstrapFewShotWithRandomSearch` optimizer to create successful examples of the ReAct program and attempt to optimize the prompts using those constructed examples. In the future, we could try more sophisticated DSPy optimizers too, like `MIPRO`.

We'll bootstrap 20 programs that way. Examples will be bootstrapped starting from the `trainset` and optimized over our tiny `valset`. We'll evaluate later on the `devset`.

让我们使用DSPy的简单优化器`BootstrapFewShotWithRandomSearch`来创建ReAct程序的成功示例，并尝试使用这些构建的示例来优化提示。将来，我们还可以尝试更复杂的DSPy优化器，如`MIPRO`。我们将以这种方式引导20个程序。这些示例将从`trainset`开始引导，并在我们的小型`valset`上进行优化。稍后我们将在`devset`上进行评估

In [7]:
# 构造函数初始化 BootstrapFewShotWithRandomSearch 类并设置其属性。它继承自 BootstrapFewShot 类，并为随机搜索过程引入了附加属性。
# max_bootstrapped_demos=2：在少样本学习中最多使用2个引导样本。 max_labeled_demos=0：不使用任何标记样本。 num_candidate_programs=20：在随机搜索中一共会评估20个候选程序。




config = dict(max_bootstrapped_demos=2, max_labeled_demos=0, num_candidate_programs=20, num_threads=32)

tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **config)
optimized_react = tp.compile(agent, trainset=trainset, valset=valset)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 20 candidate sets.
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)



Average Metric: 8.0 / 50  (16.0): 100%|██████████| 50/50 [00:00<00:00, 447.27it/s]


Average Metric: 8.0 / 50  (16.0%)
Score: 16.0 for set: [0, 0, 0, 0, 0]
New best score: 16.0 for seed -3
Scores so far: [16.0]
Best score: 16.0
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)
Error for example in dev set: 		 not enough values to unpack (expected 2, got 1)



Average Metric: 8.0 / 50  (16.0): 100%|██████████| 50/50 [00:00<00:00, 521.17it/s]


Average Metric: 8.0 / 50  (16.0%)
Score: 16.0 for set: [0, 0, 0, 0, 0]
Scores so far: [16.0, 16.0]
Best score: 16.0


...

Average Metric: 8.0 / 25  (32.0):  48%|████▊     | 24/50 [00:59<00:52,  2.02s/it]

Error for example in dev set: 		 Request timed out.
Error for example in dev set: 		 Request timed out.


Average Metric: 8.0 / 26  (30.8):  52%|█████▏    | 26/50 [00:59<00:22,  1.06it/s]

Error for example in dev set: 		 Request timed out.


APITimeoutError: Request timed out.

In [ ]:
evaluate(optimized_react)

### 5) Zero-Shot Aggregator.

Let's now extract the best five bootstrapped ReAct programs. We'll build a simple DSPy aggregator that runs all of them then produces a final answer.
现在让我们提取最好的五个自举ReAct程序。我们将构建一个简单的DSPy聚合器，运行所有这些程序，然后产生一个最终答案。

In [7]:
from dsp.utils import flatten, deduplicate

# the best-performing five ReAct programs from the optimization process
AGENTS = [x[-1] for x in optimized_react.candidate_programs[:5]]

class Aggregator(dspy.Module):
	def __init__(self, temperature=0.0):
		"""这个类名为 Aggregator，继承自 dspy.Module。
在初始化方法中，定义了一个用于聚合的 ChainOfThought 对象 self.aggregate，以及一个温度参数 self.temperature。"""
		self.aggregate = dspy.ChainOfThought('context, question -> answer')
		self.temperature = temperature

	def forward(self, question):
		# Run all five agents with high temperature, then extract and deduplicate their observed contexts
		# 运行所有五个代理，并设置高温，然后提取和去重它们观察到的上下文
		with dspy.context(lm=turbo.copy(temperature=self.temperature)):
			preds = [agent(question=question) for agent in AGENTS]
			context = deduplicate(flatten([flatten(p.observations) for p in preds]))

		# Run the aggregation step to produce a final answer
		# 运行聚合步骤以生成最终答案
		return self.aggregate(context=context, question=question)

Let's quickly evaluate the aggregator prior to optimization.

In [8]:
aggregator = Aggregator()
evaluate(aggregator)

Average Metric: 78 / 300  (26.0): 100%|██████████| 300/300 [00:06<00:00, 45.38it/s]


,question,example_answer,gold_titles,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}",determine if both Cangzhou and Qionghai are in the Hebei province of China. We need to carefully analyze the information provided in the context to...,"No, only Cangzhou is in the Hebei province of China. Qionghai is located in Hainan province.",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","produce the answer. We know that Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season. Looking at the context provided, we...","The 2017 NHL Expansion Draft conducted by the National Hockey League filled the roster of the Vegas Golden Knights, including selecting Marc-Andre Fleury for the...",False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",identify the retired Canadian professional ice hockey player and current general manager of the Tampa Bay Lightning of the National Hockey League (NHL) whose retirement...,Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","identify the river near the Crichton Collegiate Church. We know that the church is situated in Midlothian, Scotland, and the River Esk flows through Midlothian...",The River Esk,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","produce the answer. We know from the context that Ealhswith had a son named Æthelweard in the 10th century A.D. Now, looking at the information...",King Alfred the Great,✔️ [True]


26.0

### 6) Optimized Aggregator.

In [9]:
kwargs = dict(max_bootstrapped_demos=2, max_labeled_demos=6, num_candidate_programs=10, num_threads=8)
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **kwargs)
optimized_aggregator = tp.compile(aggregator, trainset=trainset, valset=valset)

Average Metric: 30 / 50  (60.0): 100%|██████████| 50/50 [00:00<00:00, 148.52it/s]


In [10]:
optimized_aggregator2 = optimized_aggregator.deepcopy()
optimized_aggregator2.temperature = 0.7

evaluate(optimized_aggregator2)

Average Metric: 180 / 300  (60.0): 100%|██████████| 300/300 [00:07<00:00, 42.10it/s]


,question,example_answer,gold_titles,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","produce the answer. From the context, we know that Cangzhou is a prefecture-level city in eastern Hebei province, while Qionghai is one of the seven...",no,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","produce the answer. From the context, we know that Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season. The draft that...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",produce the answer. We know from the context that Steve Yzerman is a Canadian retired professional ice hockey player and the current general manager of...,Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","produce the answer. We know that Crichton Collegiate Church is located in Midlothian, Scotland, near the hamlet of Crichton. Since it is close to Edinburgh,...",River Esk,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","produce the answer. From the context, we know that Ealhswith was the wife of King Alfred the Great. Therefore, in the 10th Century A.D., Ealhswith...",King Alfred the Great,✔️ [True]


60.0